### Exploring Hail ploidy inference for DRAGMAP alignments

Ploidy in DRAGMAP samples was inferred incorrectly on chrY for multiple samples, usng gnomad's annotate_sex 

For example, these 3:

  * CPG1016:  DRAGMAP ploidy: XXY, BWA-MEM ploidiy: XX
  * CPG1032:  DRAGMAP ploidy: XXY, BWA-MEM ploidiy: XX
  * CPG10132: DRAGMAP ploidy: XX,  BWA-MEM ploidiy: XX

The ploidy is inferred based on coverage thresholds on non-par chrY regions.
Coverage is calculated by averaging DP of each variant and reference block.
To figure out which variants or blocks contribute to the coverage disparities,
summarising coverage on chrY with difference precision: 
every 10 million, every million, etc.

First, extracting POS, END, DP for each GVCF regord into a TSV file:

```sh
cd /Users/vlad/CPG/nagim-sexy
mkdir chry_gvcfs
cd chry_gvcfs
for ID in CPG1016 CPG1032 CPG10132; do
  bcftools view -t chrY gs://cpg-tob-wgs-main/gvcf/bwamem/$ID.g.vcf.gz -Oz -o $ID-bwamem.g.vcf.gz
  bcftools view -t chrY gs://cpg-tob-wgs-main/gvcf/nagim/$ID.g.vcf.gz -Oz -o $ID-nagim.g.vcf.gz
  tabix $ID-bwamem.g.vcf.gz
  tabix $ID-nagim.g.vcf.gz
done
for ID in CPG1016 CPG1032 CPG10132; do
  bcftools view -t chrY $ID-bwamem.g.vcf.gz | bcftools query -f '%POS\t%END\t[%DP]\n' > $ID-bwamem-chry.tsv
  bcftools view -t chrY $ID-nagim.g.vcf.gz | bcftools query -f '%POS\t%END\t[%DP]\n' > $ID-nagim-chry.tsv
done
```

In [2]:
from dataclasses import dataclass

# Parsing those lines into lists

def parse(fpath) -> list:
  lines = []
  with open(fpath) as f:
    for line in f:
      pos, end, dp = line.strip().split()
      pos = int(pos)
      end = int(end)
      if dp == '.':
        dp = 0
      dp = int(dp)
      lines.append((pos, end, dp))
  assert lines
  return lines

@dataclass
class Sample:
  name: str
  bwamem_lines: list
  nagim_lines: list

samples = [
  Sample(
    s,
    parse(f'/Users/vlad/CPG/nagim-sexy/chry_gvcfs/{s}-bwamem-chry.tsv'),
    parse(f'/Users/vlad/CPG/nagim-sexy/chry_gvcfs/{s}-nagim-chry.tsv'),
  )
  for s in ['CPG1016', 'CPG1032', 'CPG10132']
]

In [3]:
"""
Summarising DP for each sample
"""

def tally(lines, region_size):
  start_by_batch = {}
  end_by_batch = {}
  # sum of lengths of regions defined in VCF. We need to calculate it instead
  # of just taking the chromosome length, because some regions are skipped:
  size_by_batch = {}
  num_records_by_batch = {}
  sum_dp_by_batch = {}
  for (pos, end, dp) in lines:  # iterating over each variant
    k = pos // region_size  # batch number

    if k not in start_by_batch:
      start_by_batch[k] = pos
    end_by_batch[k] = end
  
    if k not in size_by_batch:
      size_by_batch[k] = 0
    size_by_batch[k] += (end - pos + 1)

    if k not in sum_dp_by_batch:
      sum_dp_by_batch[k] = 0
    sum_dp_by_batch[k] += dp * (end - pos + 1)
      
    if k not in num_records_by_batch:
      num_records_by_batch[k] = 0
    num_records_by_batch[k] += 1
  
  line_by_batch = {}  
  for k, sum_dp in sum_dp_by_batch.items():
    mean_dp = sum_dp / size_by_batch[k]
    line_by_batch[k] = (
      f'{start_by_batch[k]}-{end_by_batch[k]}: '
      f'{mean_dp:.2f} / { size_by_batch[k]//1000}k [{num_records_by_batch[k]}]'
    )
  return line_by_batch

print('---')
for sample in samples:
  print(f'*' * 60)
  print(f'* {sample.name} *')
  for region_size in [100_000, 1_000_000, 10_000_000, 100_000_000]:
    print(f'Summarizing {sample.name} on the level of {region_size} (NAGIM, BWAMEM):')
    nagim_by_batch = tally(sample.nagim_lines, region_size)
    bwamem_by_batch = tally(sample.bwamem_lines, region_size)
    max_batch_num = max([max(nagim_by_batch), max(bwamem_by_batch)])
    for batch_num in range(0, max_batch_num + 1):
      nagim_line = nagim_by_batch.get(batch_num, '')
      bwamem_line = bwamem_by_batch.get(batch_num, '')
      if not nagim_line or not bwamem_line:
        continue
      linestart = f'{batch_num}: {nagim_line}'
      print(f'{linestart}{" "*(45-len(linestart))}{bwamem_line}')
    print()

---
************************************************************
* CPG1016 *
Summarizing CPG1016 on the level of 100000 (NAGIM, BWAMEM):
27: 2781480-3050594: 3.00 / 269k [1]         2781480-3050594: 0.00 / 269k [1]
30: 3050595-3101833: 4.72 / 51k [140]        3050595-3101833: 0.33 / 51k [136]
31: 3101834-3200895: 3.43 / 99k [246]        3101834-3200898: 0.26 / 99k [135]
32: 3200896-3303962: 3.25 / 103k [169]       3200899-3303962: 0.23 / 103k [171]
33: 3303963-3403250: 3.78 / 99k [475]        3303963-3403250: 0.62 / 99k [379]
34: 3403251-3509251: 4.30 / 106k [383]       3403251-3509260: 2.78 / 106k [188]
35: 3509252-3606938: 6.16 / 97k [1171]       3509261-3606938: 2.59 / 97k [1128]
36: 3606939-3700228: 4.49 / 93k [214]        3606939-3702892: 0.29 / 95k [180]
37: 3700229-3846459: 4.36 / 146k [392]       3702893-3827324: 0.34 / 124k [292]
38: 3846460-3924363: 4.49 / 77k [14]         3827325-3924381: 0.01 / 97k [10]
39: 3924364-4034142: 5.07 / 109k [29]        3924382-4034386: 0.02 / 11

27: 2781480-3050602: 3.00 / 269k [1]         2781480-3050602: 0.00 / 269k [1]
30: 3050603-3100051: 4.80 / 49k [136]        3050603-3100124: 0.73 / 49k [168]
31: 3100052-3200848: 3.02 / 100k [172]       3100125-3200848: 0.31 / 100k [132]
32: 3200849-3304102: 4.55 / 103k [324]       3200849-3304335: 0.35 / 103k [248]
33: 3304103-3403250: 3.46 / 99k [439]        3304336-3403250: 0.51 / 98k [317]
34: 3403251-3519777: 3.13 / 116k [234]       3403251-3519777: 1.40 / 116k [129]
35: 3519778-3606938: 4.21 / 87k [590]        3519778-3606938: 1.19 / 87k [575]
36: 3606939-3700271: 3.69 / 93k [188]        3606939-3702892: 0.31 / 95k [156]
37: 3700272-3833146: 4.50 / 132k [150]       3702893-3833146: 0.21 / 130k [160]
38: 3833147-3925359: 7.73 / 92k [601]        3833147-3925359: 4.08 / 92k [522]
39: 3925360-4002911: 2.27 / 77k [46]         3925360-4013978: 0.07 / 88k [30]
40: 4002912-4102372: 3.75 / 99k [228]        4013979-4121615: 0.22 / 107k [173]
41: 4102373-4220040: 3.41 / 117k [473]       4121

0: 2781480-56887902: 20.77 / 23636k [68434]  2781480-56887902: 0.99 / 23642k [63455]

************************************************************
* CPG10132 *
Summarizing CPG10132 on the level of 100000 (NAGIM, BWAMEM):
27: 2781480-3050714: 4.00 / 269k [1]         2781480-3050716: 0.00 / 269k [1]
30: 3050715-3100071: 3.53 / 49k [145]        3050717-3100124: 0.11 / 49k [42]
31: 3100072-3200905: 2.07 / 100k [82]        3100125-3200936: 0.09 / 100k [32]
32: 3200906-3304335: 2.98 / 103k [113]       3200937-3304335: 0.11 / 103k [76]
33: 3304336-3403304: 3.29 / 98k [165]        3304336-3403304: 0.22 / 98k [98]
34: 3403305-3501859: 2.94 / 98k [105]        3403305-3501859: 0.19 / 98k [69]
35: 3501860-3678018: 3.11 / 176k [101]       3501860-3678018: 0.08 / 176k [81]
36: 3678019-3700271: 3.61 / 22k [43]         3678019-3702858: 0.11 / 24k [26]
37: 3700272-3833245: 2.34 / 132k [110]       3702859-3833245: 0.11 / 130k [75]
38: 3833246-3961546: 5.15 / 128k [558]       3833246-3961604: 1.10 / 128k

0: 2781480-56887902: 2.84 / 23636k [54183]   2781480-56887902: 0.70 / 23642k [50571]



We can see that in CPG1016 and CPG1032, regions around 15363065-16027941 and 15882979-16635242 
have a very high coverage. In CPG1016, only 16 GVCF records fall into this region,
and in CPG1032, there are just 2. Checking which exact GVCF records are 
responsible for the high coverage:

In [5]:
print('---')
print('NAGIM regions with DP>500:')
for sample in samples:
  print(f'* {sample.name} *')
  for (pos, end, dp) in sample.nagim_lines:
    if 15363065 < pos < 16635242 and dp > 500:
      print(f'{pos}-{end} DP={dp}')

---
NAGIM regions with DP>500:
* CPG1016 *
15363283-15886436 DP=712
15886743-16027941 DP=697
* CPG1032 *
15882980-16635242 DP=556
* CPG10132 *


Just 2 and 1 very long reference blocks correspondingly

Now looking at coverage in detail in one of those regions (15886743-16027941 DP=697 in CPG1016).

Calculating coverage distribution with samtools:
  
```
samtools view gs://cpg-tob-wgs-main/cram/nagim/CPG1016.cram chrY:15886743-16027941 -Obam > CPG1016-nagim-chrY-15886743-16027941.cram
samtools stats CPG1016-nagim-chrY-15886743-16027941.cram > CPG1016-nagim-chrY-15886743-16027941.cram.stats.txt
```

In [6]:
from pprint import pprint
with open('/Users/vlad/CPG/nagim-sexy/CPG1016-nagim-chrY-15886743-16027941.cram.stats.txt') as f:
  sites_by_cov = dict()
  for line in f:
    if line.startswith('COV'):
      name, rng, cov, sites = line.strip().split('\t')
      sites_by_cov[int(cov)] = int(sites)

print(sites_by_cov)

{1: 9311, 2: 1655, 3: 588, 4: 535, 5: 231, 6: 204, 7: 178, 8: 156, 9: 81, 10: 29, 11: 26, 12: 55, 13: 21, 14: 57, 15: 121, 16: 70, 17: 68, 18: 46, 19: 14, 20: 106, 21: 8, 22: 32, 23: 4, 24: 5, 25: 26, 26: 24, 27: 45, 28: 16, 29: 22, 30: 15, 31: 2, 32: 4, 33: 2, 34: 1, 35: 17, 36: 1, 39: 1, 40: 1, 42: 3, 46: 1, 47: 2, 48: 1, 49: 1, 51: 1, 53: 1, 54: 21, 57: 1, 58: 2, 61: 1, 63: 1, 64: 2, 65: 1, 66: 1, 67: 3, 68: 3, 69: 2, 144: 1, 147: 1, 791: 1, 1000: 62}


We can see that most of that region has a relatively low coverage. 
Only 62 bases have coverage >1000 base pairs. Looking at that region in IGV,
it's a 62-long streatch of repeated T, attracting 20k reads:

![](igv.png)

Now we should probably try to understand:
- why a small repetitive region contributed to the average DP of a long 
region (I guess that's because of our really coarse quality binning scheme), 
- and why this repetitive region attracted reads in DRAGMAP, but not in BWA-MEM.